In [ ]:
import numpy as np
import gdspy
import matplotlib.pyplot as plt

%matplotlib inline
from functools import partial

In [ ]:
%load_ext autoreload
%autoreload 2
import snakenmake
from snakenmake import *

In [ ]:
def plot_cell(cell, exclude=(2,)):
    # FROM: https://github.com/heitzmann/gdspy/issues/42
    poly_dict = cell.get_polygons(by_spec=True)
    plt.figure(figsize=(20, 12))
    for layer_datatype, polys in poly_dict.items():
        if layer_datatype[0] in exclude:
            continue
        for poly in polys:
            plt.fill(*poly.T, lw=0.5, ec="r", fc="none")
    plt.axes().set_aspect("equal", "datalim")


def write_gds(main_cell, filename, unit=1.0e-6, precision=1.0e-9):
    writer = gdspy.GdsWriter(filename, unit=unit, precision=precision)
    for cell in [main_cell] + list(main_cell.get_dependencies(True)):
        writer.write_cell(cell)
    writer.close()

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
chip_dims = np.array([23e3, 13e3])

In [ ]:
fov = 6.5 * 2048 / 40
fov

In [ ]:
23e3 * 13e3 / fov**2

In [ ]:
from snakenmake import *

main_cell = wafer(
    [chip("Big Snake S3 FC90 L35 W1.5 LG20 TS2", snake, split=3, draw_trenches=True)]
    * 3,
    "Big snakes\n180503",
)
write_gds(main_cell, "bigsnakes2.gds")

In [ ]:
from snakenmake import *

main_cell = wafer(
    [
        chip(
            "Big Snake S3 FC{fc} L35 W1.5 TS{ts} LG20".format(fc=fc, ts=ts),
            snake,
            split=3,
            feeding_channel_width=fc,
            trench_spacing=ts,
            draw_trenches=True,
        )
        for fc in (90, 65)
        for ts in (1, 1.5, 2)
    ],
    "Big Snakes \n BS/JQS 180508",
)
write_gds(main_cell, "bigsnakes2.gds")